In [20]:
import os
import tempfile
import numpy as np
import pandas as pd
import cv2
import mediapipe as mp
import matplotlib.pyplot as plt
import pickle
import math
import catboost as cb

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

In [10]:
train_dir = 'data/train'
# 비디오 파일 목록과 태그를 포함하는 리스트를 만드는 함수
def create_data_list(data_dir):
    data_list = []
    for item in os.listdir(data_dir):
        item_path = os.path.join(data_dir, item)
        if os.path.isdir(item_path):
            for file_name in os.listdir(item_path):
                if file_name.endswith('.jpg'):
                    data_list.append((item, str(data_dir+'/'+item)+'/'+file_name))
    return data_list

train_list = create_data_list(train_dir)
train_df = pd.DataFrame(data=train_list, columns=['tag', 'image_name'])
train_df.to_csv('train_data.csv', encoding='utf-8-sig', index=False)
print(f"학습 데이터: {len(train_df)}")

학습 데이터: 0


In [11]:
# 손가락 각도 계산 함수
def calculate_angles(hand_landmarks, image_shape):
    joint = np.zeros((21, 3))
    for j, lm in enumerate(hand_landmarks.landmark):
        joint[j] = [lm.x * image_shape[1], lm.y * image_shape[0], lm.z]
        
    # 벡터 계산
    v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19],:]
    v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],:]
    v = v2 - v1
    v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

    # 각도 계산
    angle = np.arccos(np.einsum('nt,nt->n', v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:]))
    angle = np.degrees(angle)
    
    # NaN 값이 있을 경우 0으로 대체
    angle = np.nan_to_num(angle)
    
    return angle, joint.flatten()

In [12]:
def calculate_pose_angles(pose_landmarks, image_shape):
    joint = np.zeros((33, 3))
    for j, lm in enumerate(pose_landmarks.landmark):
        joint[j] = [lm.x * image_shape[1], lm.y * image_shape[0], lm.z]

    # 팔 각도 계산: 어깨(11, 12), 팔꿈치(13, 14), 손목(15, 16) 랜드마크 사용
    v1 = joint[[11, 13, 12, 14], :]  # 어깨와 팔꿈치
    v2 = joint[[13, 15, 14, 16], :]  # 팔꿈치와 손목
    v = v2 - v1
    v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

    # 양팔의 각도 계산
    arm_angles = np.arccos(np.einsum('nt,nt->n', v[[0, 2], :], v[[1, 3], :]))
    arm_angles = np.degrees(arm_angles)

    return arm_angles

In [5]:
df_data = pd.DataFrame()

In [6]:
def create_training_data_from_image(image_path, label):
    global df_data
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(static_image_mode=True, max_num_hands=2, min_detection_confidence=0.5)
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)
    
    # 이미지 파일 불러오기
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result = hands.process(img)
    pose_result = pose.process(img)

    landmark_data = []
    hand_data_row = [0] * 30
    arm_data_row = [0] * 2
    right_hand_data_row = [0] * 63
    left_hand_data_row = [0] * 63
    
    if result.multi_hand_landmarks:
        for i, hand_landmarks in enumerate(result.multi_hand_landmarks):
            hand_type = result.multi_handedness[hand_idx].classification[0].label
            
            angles, joint_coords = calculate_angles(hand_landmarks, img.shape[:2])
            hand_data = angles.tolist() + joint_coords.tolist()
            
            # 손 타입에 따라 데이터 저장
            if hand_type == 'Right':
                right_hand_data_row = hand_data
            elif hand_type == 'Left':
                left_hand_data_row = hand_data
            
    if pose_result.pose_landmarks:
        arm_angles = calculate_pose_angles(pose_result.pose_landmarks, img.shape[:2])
        arm_data_row = arm_angles.tolist()
        for idx in [11, 13, 15, 12, 14, 16]:
            landmark = pose_result.pose_landmarks.landmark[idx]
            landmark_data.extend([landmark.x, landmark.y, landmark.z])

    df_data = pd.concat([df_data, pd.DataFrame([right_hand_data_row + left_hand_data_row + arm_data_row + landmark_data + [label]])], ignore_index=True)

    return df_data

In [7]:
# 특정 폴더의 모든 이미지를 처리하고 CSV 파일로 저장하는 함수
def process_folder(folder_path, label):
    df_folder = pd.DataFrame()

    # 폴더 내 모든 이미지 처리
    for image_name in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image_name)
        df_folder = create_training_data_from_image(image_path, label)

    # 컬럼 이름 설정 (30개의 각도 + 1개의 라벨)
    right_hand_columns = [f'right_hand_{i}_{axis}' for i in range(1, 22) for axis in ['x', 'y', 'z']]
    left_hand_columns = [f'left_hand_{i}_{axis}' for i in range(1, 22) for axis in ['x', 'y', 'z']]
    arm_angle_columns = ['arm_1', 'arm_2']
    other_landmark_columns = [f'{idx}_{axis}' for idx in [11, 13, 15, 12, 14, 16] for axis in ['x', 'y', 'z']]
    
    all_columns = right_hand_columns + left_hand_columns + arm_angle_columns + other_landmark_columns + ['label']
    df_folder.columns = all_columns
    df_folder.to_csv(f'{label}_data.csv', index=False)

In [8]:
# 'data/train' 폴더 내 모든 하위 폴더를 순회
base_folder = 'data/train'
for label_folder in os.listdir(base_folder):
    folder_path = base_folder+"/"+label_folder
    if os.path.isdir(folder_path):
        print(folder_path)
        process_folder(folder_path, label_folder)

data/train/where


In [21]:
df_data = pd.read_csv("final_training_data.csv")

In [22]:
df_data

,angle_0,angle_1,angle_2,angle_3,angle_4,angle_5,angle_6,angle_7,angle_8,angle_9,...,12_x,12_y,12_z,14_x,14_y,14_z,16_x,16_y,16_z,label
0,17.626396,5.989130,10.317576,15.149633,1.447276,0.320342,4.348895,1.877061,0.322290,14.482118,...,0.304181,0.269859,-0.348680,0.166155,0.444097,-0.433554,0.386356,0.487853,-0.751229,belly
1,15.812271,6.516015,11.230466,20.231538,1.977787,1.692256,4.297915,3.216543,0.189223,16.986444,...,0.308368,0.272443,-0.268009,0.169883,0.445997,-0.414924,0.379054,0.481162,-0.839744,belly
2,15.429004,5.735022,10.687926,25.062960,0.247285,0.075435,6.428805,1.805776,1.141801,15.441685,...,0.311658,0.272051,-0.334071,0.170422,0.444774,-0.460341,0.380478,0.479141,-0.846146,belly
3,17.752119,5.297128,9.918691,23.915642,0.235788,0.838819,6.195439,2.406301,1.803850,15.783006,...,0.313875,0.270783,-0.190262,0.164895,0.441304,-0.319333,0.378377,0.476458,-0.652272,belly
4,15.287326,4.806262,8.508105,24.556346,0.314952,1.951877,5.933526,0.781696,2.717788,15.666043,...,0.316910,0.266524,-0.211217,0.170624,0.440041,-0.354612,0.381129,0.475872,-0.706072,belly
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1662,13.164771,15.750716,15.800542,25.561584,3.531353,2.593089,22.276562,5.811919,0.717926,14.740576,...,0.381963,0.314128,-0.110261,0.363492,0.467962,-0.333105,0.369171,0.466111,-0.899335,where
1663,15.836760,14.802287,17.196987,25.543790,4.340256,2.709850,23.225966,7.479944,1.613820,16.325354,...,0.365274,0.319890,-0.164540,0.345364,0.474336,-0.373707,0.357359,0.468207,-0.886312,where
1664,12.810530,16.397194,14.184675,24.915383,6.833520,3.392613,25.238710,8.054143,2.581491,17.548025,...,0.360834,0.336194,-0.093453,0.338578,0.493374,-0.232550,0.351179,0.479158,-0.677563,where
1665,12.780273,15.659962,14.740101,25.321299,3.311745,3.496567,22.705196,6.292655,0.979550,16.233037,...,0.364342,0.358288,-0.104552,0.342934,0.499110,-0.225479,0.355064,0.496805,-0.642289,where


In [23]:
train_df, test_df = train_test_split(df_data, test_size=0.2, random_state=42)

In [24]:
train_df

,angle_0,angle_1,angle_2,angle_3,angle_4,angle_5,angle_6,angle_7,angle_8,angle_9,...,12_x,12_y,12_z,14_x,14_y,14_z,16_x,16_y,16_z,label
1107,21.604858,5.282122,14.396670,38.831269,118.212730,18.667658,62.339727,172.385791,4.203914,7.226060,...,0.420467,0.393429,-0.179487,0.398885,0.525159,-0.204885,0.390743,0.589146,-0.596701,sick
266,20.533895,18.703311,1.526157,4.965500,32.674505,17.562370,11.542071,34.050281,19.624584,8.095876,...,0.367064,0.558114,-0.151171,0.118354,0.690681,-0.586773,0.255760,0.511223,-1.087224,child
148,22.065187,0.314356,13.352421,24.015674,2.900800,3.671718,4.530293,1.102121,4.101610,11.226040,...,0.260899,0.295553,-0.183296,0.120585,0.460280,-0.343750,0.315324,0.524766,-0.690026,belly
433,122.269201,9.651234,12.984581,14.750763,7.068793,3.691028,2.800100,10.413831,3.557138,35.021045,...,0.304040,0.432958,-0.286579,0.262094,0.583945,-0.392036,0.364870,0.608340,-0.829084,down
874,28.246121,23.411004,70.520696,13.505887,1.391123,8.731151,23.865756,138.251439,28.893506,41.611859,...,0.409848,0.544603,-0.026012,0.297086,0.710932,-0.313925,0.356374,0.537299,-0.690937,report
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1130,5.545419,32.949791,32.025721,18.174258,80.064898,1.720336,67.114700,162.936914,5.839856,73.010891,...,0.484574,0.354842,-0.175845,0.469709,0.511857,-0.090234,0.499060,0.551349,-0.375247,sick
1294,11.155837,18.871203,6.438106,5.921426,75.714444,25.195751,19.166851,16.769235,3.134082,18.729527,...,0.460942,0.449786,-0.117826,0.380923,0.627277,-0.325617,0.400803,0.522288,-0.864194,toilet
860,22.956300,19.611363,66.948342,9.778421,1.723309,11.495936,33.055762,132.612371,36.303627,48.822108,...,0.449921,0.485010,-0.006212,0.344544,0.645043,-0.304601,0.407434,0.476474,-0.676927,report
1459,45.205475,28.338274,8.103307,5.808063,4.153102,5.714535,7.533821,7.981591,4.991177,15.758266,...,0.371720,0.378859,-0.260999,0.252231,0.538301,-0.304720,0.401507,0.576320,-0.691062,wallet


In [25]:
X_train = train_df.drop('label', axis =1)
y_train = train_df['label']

In [26]:
X_test = test_df.drop('label', axis =1)
y_test = test_df['label']

In [27]:
lgbm_model = LGBMClassifier(random_state=200, n_jobs=-1, reg_alpha=0.9, reg_lambda=0.2, n_estimators=200)
lgbm_model.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001798 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 11490
[LightGBM] [Info] Number of data points in the train set: 1333, number of used features: 50
[LightGBM] [Info] Start training from score -2.275206
[LightGBM] [Info] Start training from score -2.304838
[LightGBM] [Info] Start training from score -2.366874
[LightGBM] [Info] Start training from score -2.335375
[LightGBM] [Info] Start training from score -2.343157
[LightGBM] [Info] Start training from score -2.304838
[LightGBM] [Info] Start training from score -2.297348
[LightGBM] [Info] Start training from score -2.327653
[LightGBM] [Info] Start training from score -2.253545
[LightGBM] [Info] Start training from score -2.225374
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

LGBMClassifier(n_estimators=200, n_jobs=-1, random_state=200, reg_alpha=0.9,
               reg_lambda=0.2)

In [28]:
result = cross_val_score(lgbm_model,
                          X_test,
                          y_test,
                          cv = 5)
result.mean()

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000286 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3705
[LightGBM] [Info] Number of data points in the train set: 267, number of used features: 50
[LightGBM] [Info] Start training from score -2.368373
[LightGBM] [Info] Start training from score -2.255044
[LightGBM] [Info] Start training from score -2.090741
[LightGBM] [Info] Start training from score -2.186051
[LightGBM] [Info] Start training from score -2.255044
[LightGBM] [Info] Start training from score -2.291412
[LightGBM] [Info] Start training from score -2.409195
[LightGBM] [Info] Start training from score -2.121513
[LightGBM] [Info] Start training from score -2.591516
[LightGBM] [Info] Start training from score -2.591516
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000269 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3705
[LightGBM] [Info] Number of data points in the train set: 267, number of used features: 50
[LightGBM] [Info] Start training from score -2.329152
[LightGBM] [Info] Start training from score -2.255044
[LightGBM] [Info] Start training from score -2.060888
[LightGBM] [Info] Start training from score -2.219953
[LightGBM] [Info] Start training from score -2.291412
[LightGBM] [Info] Start training from score -2.291412
[LightGBM] [Info] Start training from score -2.409195
[LightGBM] [Info] Start training from score -2.121513
[LightGBM] [Info] Start training from score -2.591516
[LightGBM] [Info] Start training from score -2.591516
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

0.9940298507462686

In [29]:
lgbm_model.booster_.save_model('lgbm_model.txt')

In [30]:
cb_model = CatBoostClassifier()
cb_model.fit(X_train, y_train)

Learning rate set to 0.080295
0:	learn: 1.9295581	total: 205ms	remaining: 3m 24s
1:	learn: 1.6393919	total: 248ms	remaining: 2m 3s
2:	learn: 1.4257817	total: 289ms	remaining: 1m 36s
3:	learn: 1.2601793	total: 331ms	remaining: 1m 22s
4:	learn: 1.1295543	total: 376ms	remaining: 1m 14s
5:	learn: 1.0191256	total: 417ms	remaining: 1m 9s
6:	learn: 0.9240596	total: 460ms	remaining: 1m 5s
7:	learn: 0.8353752	total: 501ms	remaining: 1m 2s
8:	learn: 0.7622939	total: 542ms	remaining: 59.7s
9:	learn: 0.6955086	total: 584ms	remaining: 57.8s
10:	learn: 0.6393567	total: 626ms	remaining: 56.3s
11:	learn: 0.5868504	total: 668ms	remaining: 55s
12:	learn: 0.5412091	total: 710ms	remaining: 53.9s
13:	learn: 0.5040494	total: 752ms	remaining: 53s
14:	learn: 0.4676160	total: 795ms	remaining: 52.2s
15:	learn: 0.4332273	total: 837ms	remaining: 51.5s
16:	learn: 0.4024276	total: 879ms	remaining: 50.8s
17:	learn: 0.3749239	total: 923ms	remaining: 50.3s
18:	learn: 0.3505257	total: 966ms	remaining: 49.9s
19:	learn: 

162:	learn: 0.0116527	total: 7.2s	remaining: 37s
163:	learn: 0.0116108	total: 7.25s	remaining: 37s
164:	learn: 0.0115526	total: 7.29s	remaining: 36.9s
165:	learn: 0.0114792	total: 7.34s	remaining: 36.9s
166:	learn: 0.0113359	total: 7.38s	remaining: 36.8s
167:	learn: 0.0112270	total: 7.43s	remaining: 36.8s
168:	learn: 0.0111582	total: 7.47s	remaining: 36.8s
169:	learn: 0.0110321	total: 7.52s	remaining: 36.7s
170:	learn: 0.0109039	total: 7.56s	remaining: 36.7s
171:	learn: 0.0108557	total: 7.61s	remaining: 36.6s
172:	learn: 0.0107427	total: 7.65s	remaining: 36.6s
173:	learn: 0.0106560	total: 7.7s	remaining: 36.5s
174:	learn: 0.0105082	total: 7.74s	remaining: 36.5s
175:	learn: 0.0104480	total: 7.79s	remaining: 36.5s
176:	learn: 0.0103471	total: 7.83s	remaining: 36.4s
177:	learn: 0.0102923	total: 7.88s	remaining: 36.4s
178:	learn: 0.0102266	total: 7.92s	remaining: 36.3s
179:	learn: 0.0101405	total: 7.96s	remaining: 36.3s
180:	learn: 0.0100180	total: 8.01s	remaining: 36.2s
181:	learn: 0.0099

322:	learn: 0.0048610	total: 14.4s	remaining: 30.2s
323:	learn: 0.0048445	total: 14.5s	remaining: 30.2s
324:	learn: 0.0048258	total: 14.5s	remaining: 30.1s
325:	learn: 0.0047987	total: 14.5s	remaining: 30.1s
326:	learn: 0.0047804	total: 14.6s	remaining: 30s
327:	learn: 0.0047716	total: 14.6s	remaining: 30s
328:	learn: 0.0047576	total: 14.7s	remaining: 29.9s
329:	learn: 0.0047462	total: 14.7s	remaining: 29.9s
330:	learn: 0.0047278	total: 14.8s	remaining: 29.9s
331:	learn: 0.0047041	total: 14.8s	remaining: 29.8s
332:	learn: 0.0046757	total: 14.9s	remaining: 29.8s
333:	learn: 0.0046686	total: 14.9s	remaining: 29.7s
334:	learn: 0.0046608	total: 14.9s	remaining: 29.7s
335:	learn: 0.0046436	total: 15s	remaining: 29.6s
336:	learn: 0.0046202	total: 15s	remaining: 29.6s
337:	learn: 0.0046024	total: 15.1s	remaining: 29.5s
338:	learn: 0.0045890	total: 15.1s	remaining: 29.5s
339:	learn: 0.0045654	total: 15.2s	remaining: 29.5s
340:	learn: 0.0045426	total: 15.2s	remaining: 29.4s
341:	learn: 0.004527

482:	learn: 0.0029003	total: 21.6s	remaining: 23.1s
483:	learn: 0.0028947	total: 21.6s	remaining: 23.1s
484:	learn: 0.0028887	total: 21.7s	remaining: 23s
485:	learn: 0.0028801	total: 21.7s	remaining: 23s
486:	learn: 0.0028722	total: 21.8s	remaining: 22.9s
487:	learn: 0.0028661	total: 21.8s	remaining: 22.9s
488:	learn: 0.0028524	total: 21.9s	remaining: 22.9s
489:	learn: 0.0028453	total: 21.9s	remaining: 22.8s
490:	learn: 0.0028398	total: 22s	remaining: 22.8s
491:	learn: 0.0028304	total: 22s	remaining: 22.7s
492:	learn: 0.0028253	total: 22s	remaining: 22.7s
493:	learn: 0.0028215	total: 22.1s	remaining: 22.6s
494:	learn: 0.0028145	total: 22.1s	remaining: 22.6s
495:	learn: 0.0028029	total: 22.2s	remaining: 22.5s
496:	learn: 0.0028004	total: 22.2s	remaining: 22.5s
497:	learn: 0.0027920	total: 22.3s	remaining: 22.4s
498:	learn: 0.0027801	total: 22.3s	remaining: 22.4s
499:	learn: 0.0027783	total: 22.4s	remaining: 22.4s
500:	learn: 0.0027707	total: 22.4s	remaining: 22.3s
501:	learn: 0.0027635	

643:	learn: 0.0020429	total: 28.8s	remaining: 15.9s
644:	learn: 0.0020388	total: 28.9s	remaining: 15.9s
645:	learn: 0.0020371	total: 28.9s	remaining: 15.8s
646:	learn: 0.0020333	total: 28.9s	remaining: 15.8s
647:	learn: 0.0020270	total: 29s	remaining: 15.8s
648:	learn: 0.0020203	total: 29.1s	remaining: 15.7s
649:	learn: 0.0020133	total: 29.1s	remaining: 15.7s
650:	learn: 0.0020123	total: 29.1s	remaining: 15.6s
651:	learn: 0.0020079	total: 29.2s	remaining: 15.6s
652:	learn: 0.0020025	total: 29.2s	remaining: 15.5s
653:	learn: 0.0020014	total: 29.3s	remaining: 15.5s
654:	learn: 0.0019955	total: 29.3s	remaining: 15.4s
655:	learn: 0.0019945	total: 29.4s	remaining: 15.4s
656:	learn: 0.0019912	total: 29.4s	remaining: 15.4s
657:	learn: 0.0019880	total: 29.5s	remaining: 15.3s
658:	learn: 0.0019863	total: 29.5s	remaining: 15.3s
659:	learn: 0.0019806	total: 29.5s	remaining: 15.2s
660:	learn: 0.0019791	total: 29.6s	remaining: 15.2s
661:	learn: 0.0019736	total: 29.6s	remaining: 15.1s
662:	learn: 0.

803:	learn: 0.0015690	total: 36s	remaining: 8.78s
804:	learn: 0.0015648	total: 36s	remaining: 8.73s
805:	learn: 0.0015639	total: 36.1s	remaining: 8.69s
806:	learn: 0.0015610	total: 36.1s	remaining: 8.64s
807:	learn: 0.0015600	total: 36.2s	remaining: 8.6s
808:	learn: 0.0015588	total: 36.2s	remaining: 8.55s
809:	learn: 0.0015563	total: 36.3s	remaining: 8.51s
810:	learn: 0.0015548	total: 36.3s	remaining: 8.46s
811:	learn: 0.0015527	total: 36.4s	remaining: 8.42s
812:	learn: 0.0015501	total: 36.4s	remaining: 8.37s
813:	learn: 0.0015488	total: 36.4s	remaining: 8.33s
814:	learn: 0.0015476	total: 36.5s	remaining: 8.28s
815:	learn: 0.0015455	total: 36.5s	remaining: 8.24s
816:	learn: 0.0015419	total: 36.6s	remaining: 8.19s
817:	learn: 0.0015388	total: 36.6s	remaining: 8.15s
818:	learn: 0.0015384	total: 36.7s	remaining: 8.1s
819:	learn: 0.0015368	total: 36.7s	remaining: 8.06s
820:	learn: 0.0015349	total: 36.8s	remaining: 8.01s
821:	learn: 0.0015329	total: 36.8s	remaining: 7.97s
822:	learn: 0.0015

964:	learn: 0.0012751	total: 43.2s	remaining: 1.57s
965:	learn: 0.0012723	total: 43.2s	remaining: 1.52s
966:	learn: 0.0012714	total: 43.3s	remaining: 1.48s
967:	learn: 0.0012709	total: 43.3s	remaining: 1.43s
968:	learn: 0.0012704	total: 43.4s	remaining: 1.39s
969:	learn: 0.0012678	total: 43.4s	remaining: 1.34s
970:	learn: 0.0012653	total: 43.5s	remaining: 1.3s
971:	learn: 0.0012635	total: 43.5s	remaining: 1.25s
972:	learn: 0.0012631	total: 43.5s	remaining: 1.21s
973:	learn: 0.0012618	total: 43.6s	remaining: 1.16s
974:	learn: 0.0012610	total: 43.6s	remaining: 1.12s
975:	learn: 0.0012581	total: 43.7s	remaining: 1.07s
976:	learn: 0.0012571	total: 43.7s	remaining: 1.03s
977:	learn: 0.0012559	total: 43.8s	remaining: 984ms
978:	learn: 0.0012546	total: 43.8s	remaining: 940ms
979:	learn: 0.0012524	total: 43.9s	remaining: 895ms
980:	learn: 0.0012506	total: 43.9s	remaining: 850ms
981:	learn: 0.0012492	total: 43.9s	remaining: 805ms
982:	learn: 0.0012481	total: 44s	remaining: 761ms
983:	learn: 0.0

In [31]:
result = cross_val_score(cb_model,
                          X_test,
                          y_test,
                          cv = 5)
result.mean()

Learning rate set to 0.073973
0:	learn: 2.1219790	total: 31ms	remaining: 31s
1:	learn: 1.9689738	total: 63.3ms	remaining: 31.6s
2:	learn: 1.8590836	total: 96.1ms	remaining: 31.9s
3:	learn: 1.7529150	total: 129ms	remaining: 32.2s
4:	learn: 1.6288645	total: 162ms	remaining: 32.2s
5:	learn: 1.5418779	total: 195ms	remaining: 32.3s
6:	learn: 1.4291920	total: 229ms	remaining: 32.5s
7:	learn: 1.3459878	total: 264ms	remaining: 32.7s
8:	learn: 1.2613189	total: 296ms	remaining: 32.6s
9:	learn: 1.1815222	total: 328ms	remaining: 32.5s
10:	learn: 1.1142286	total: 361ms	remaining: 32.4s
11:	learn: 1.0570358	total: 393ms	remaining: 32.4s
12:	learn: 1.0054326	total: 425ms	remaining: 32.3s
13:	learn: 0.9591956	total: 459ms	remaining: 32.3s
14:	learn: 0.8994947	total: 492ms	remaining: 32.3s
15:	learn: 0.8485652	total: 524ms	remaining: 32.2s
16:	learn: 0.8095748	total: 557ms	remaining: 32.2s
17:	learn: 0.7689223	total: 589ms	remaining: 32.1s
18:	learn: 0.7315060	total: 621ms	remaining: 32.1s
19:	learn: 0

161:	learn: 0.0395603	total: 5.26s	remaining: 27.2s
162:	learn: 0.0391897	total: 5.3s	remaining: 27.2s
163:	learn: 0.0387830	total: 5.33s	remaining: 27.2s
164:	learn: 0.0383651	total: 5.36s	remaining: 27.1s
165:	learn: 0.0379743	total: 5.39s	remaining: 27.1s
166:	learn: 0.0375617	total: 5.43s	remaining: 27.1s
167:	learn: 0.0373518	total: 5.46s	remaining: 27s
168:	learn: 0.0371351	total: 5.49s	remaining: 27s
169:	learn: 0.0369903	total: 5.52s	remaining: 27s
170:	learn: 0.0367558	total: 5.55s	remaining: 26.9s
171:	learn: 0.0365316	total: 5.59s	remaining: 26.9s
172:	learn: 0.0361858	total: 5.62s	remaining: 26.9s
173:	learn: 0.0358369	total: 5.65s	remaining: 26.8s
174:	learn: 0.0356428	total: 5.68s	remaining: 26.8s
175:	learn: 0.0355274	total: 5.72s	remaining: 26.8s
176:	learn: 0.0351456	total: 5.75s	remaining: 26.7s
177:	learn: 0.0350794	total: 5.78s	remaining: 26.7s
178:	learn: 0.0347598	total: 5.82s	remaining: 26.7s
179:	learn: 0.0345854	total: 5.85s	remaining: 26.6s
180:	learn: 0.03448

324:	learn: 0.0159592	total: 10.5s	remaining: 21.8s
325:	learn: 0.0158672	total: 10.5s	remaining: 21.7s
326:	learn: 0.0158073	total: 10.5s	remaining: 21.7s
327:	learn: 0.0157366	total: 10.6s	remaining: 21.7s
328:	learn: 0.0156328	total: 10.6s	remaining: 21.6s
329:	learn: 0.0156142	total: 10.6s	remaining: 21.6s
330:	learn: 0.0155219	total: 10.7s	remaining: 21.6s
331:	learn: 0.0155046	total: 10.7s	remaining: 21.5s
332:	learn: 0.0154173	total: 10.7s	remaining: 21.5s
333:	learn: 0.0153299	total: 10.8s	remaining: 21.5s
334:	learn: 0.0152961	total: 10.8s	remaining: 21.4s
335:	learn: 0.0152250	total: 10.8s	remaining: 21.4s
336:	learn: 0.0151503	total: 10.9s	remaining: 21.4s
337:	learn: 0.0150673	total: 10.9s	remaining: 21.3s
338:	learn: 0.0150528	total: 10.9s	remaining: 21.3s
339:	learn: 0.0149929	total: 10.9s	remaining: 21.2s
340:	learn: 0.0149756	total: 11s	remaining: 21.2s
341:	learn: 0.0149201	total: 11s	remaining: 21.2s
342:	learn: 0.0148812	total: 11s	remaining: 21.2s
343:	learn: 0.0148

488:	learn: 0.0096555	total: 15.7s	remaining: 16.4s
489:	learn: 0.0096512	total: 15.7s	remaining: 16.4s
490:	learn: 0.0096475	total: 15.8s	remaining: 16.4s
491:	learn: 0.0096239	total: 15.8s	remaining: 16.3s
492:	learn: 0.0095892	total: 15.8s	remaining: 16.3s
493:	learn: 0.0095675	total: 15.9s	remaining: 16.3s
494:	learn: 0.0095387	total: 15.9s	remaining: 16.2s
495:	learn: 0.0095076	total: 15.9s	remaining: 16.2s
496:	learn: 0.0094794	total: 16s	remaining: 16.2s
497:	learn: 0.0094564	total: 16s	remaining: 16.1s
498:	learn: 0.0094369	total: 16s	remaining: 16.1s
499:	learn: 0.0094154	total: 16.1s	remaining: 16.1s
500:	learn: 0.0093853	total: 16.1s	remaining: 16s
501:	learn: 0.0093462	total: 16.1s	remaining: 16s
502:	learn: 0.0093103	total: 16.2s	remaining: 16s
503:	learn: 0.0092810	total: 16.2s	remaining: 16s
504:	learn: 0.0092501	total: 16.2s	remaining: 15.9s
505:	learn: 0.0092250	total: 16.3s	remaining: 15.9s
506:	learn: 0.0091975	total: 16.3s	remaining: 15.9s
507:	learn: 0.0091624	tota

650:	learn: 0.0067617	total: 20.9s	remaining: 11.2s
651:	learn: 0.0067469	total: 21s	remaining: 11.2s
652:	learn: 0.0067280	total: 21s	remaining: 11.2s
653:	learn: 0.0067239	total: 21s	remaining: 11.1s
654:	learn: 0.0067217	total: 21.1s	remaining: 11.1s
655:	learn: 0.0067200	total: 21.1s	remaining: 11.1s
656:	learn: 0.0067048	total: 21.1s	remaining: 11s
657:	learn: 0.0066906	total: 21.2s	remaining: 11s
658:	learn: 0.0066886	total: 21.2s	remaining: 11s
659:	learn: 0.0066743	total: 21.2s	remaining: 10.9s
660:	learn: 0.0066548	total: 21.3s	remaining: 10.9s
661:	learn: 0.0066487	total: 21.3s	remaining: 10.9s
662:	learn: 0.0066322	total: 21.3s	remaining: 10.8s
663:	learn: 0.0066246	total: 21.4s	remaining: 10.8s
664:	learn: 0.0066051	total: 21.4s	remaining: 10.8s
665:	learn: 0.0065904	total: 21.4s	remaining: 10.7s
666:	learn: 0.0065775	total: 21.5s	remaining: 10.7s
667:	learn: 0.0065655	total: 21.5s	remaining: 10.7s
668:	learn: 0.0065530	total: 21.5s	remaining: 10.6s
669:	learn: 0.0065321	to

...

381:	learn: 0.0111908	total: 12.5s	remaining: 20.3s
382:	learn: 0.0111459	total: 12.6s	remaining: 20.2s
383:	learn: 0.0111049	total: 12.6s	remaining: 20.2s
384:	learn: 0.0110609	total: 12.6s	remaining: 20.2s
385:	learn: 0.0110113	total: 12.7s	remaining: 20.1s
386:	learn: 0.0109682	total: 12.7s	remaining: 20.1s
387:	learn: 0.0109309	total: 12.7s	remaining: 20.1s
388:	learn: 0.0109087	total: 12.8s	remaining: 20s
389:	learn: 0.0108683	total: 12.8s	remaining: 20s
390:	learn: 0.0108300	total: 12.8s	remaining: 20s
391:	learn: 0.0107860	total: 12.9s	remaining: 19.9s
392:	learn: 0.0107453	total: 12.9s	remaining: 19.9s
393:	learn: 0.0107150	total: 12.9s	remaining: 19.9s
394:	learn: 0.0106800	total: 13s	remaining: 19.8s
395:	learn: 0.0106305	total: 13s	remaining: 19.8s
396:	learn: 0.0105959	total: 13s	remaining: 19.8s
397:	learn: 0.0105702	total: 13.1s	remaining: 19.8s
398:	learn: 0.0105337	total: 13.1s	remaining: 19.7s
399:	learn: 0.0105098	total: 13.1s	remaining: 19.7s
400:	learn: 0.0104801	to

541:	learn: 0.0071911	total: 17.8s	remaining: 15s
542:	learn: 0.0071741	total: 17.8s	remaining: 15s
543:	learn: 0.0071582	total: 17.9s	remaining: 15s
544:	learn: 0.0071403	total: 17.9s	remaining: 14.9s
545:	learn: 0.0071262	total: 17.9s	remaining: 14.9s
546:	learn: 0.0071082	total: 18s	remaining: 14.9s
547:	learn: 0.0070862	total: 18s	remaining: 14.8s
548:	learn: 0.0070691	total: 18s	remaining: 14.8s
549:	learn: 0.0070560	total: 18.1s	remaining: 14.8s
550:	learn: 0.0070405	total: 18.1s	remaining: 14.7s
551:	learn: 0.0070307	total: 18.1s	remaining: 14.7s
552:	learn: 0.0070167	total: 18.2s	remaining: 14.7s
553:	learn: 0.0069949	total: 18.2s	remaining: 14.6s
554:	learn: 0.0069792	total: 18.2s	remaining: 14.6s
555:	learn: 0.0069643	total: 18.2s	remaining: 14.6s
556:	learn: 0.0069452	total: 18.3s	remaining: 14.5s
557:	learn: 0.0069369	total: 18.3s	remaining: 14.5s
558:	learn: 0.0069219	total: 18.3s	remaining: 14.5s
559:	learn: 0.0069092	total: 18.4s	remaining: 14.4s
560:	learn: 0.0068961	to

700:	learn: 0.0053011	total: 23s	remaining: 9.81s
701:	learn: 0.0052970	total: 23s	remaining: 9.78s
702:	learn: 0.0052874	total: 23.1s	remaining: 9.74s
703:	learn: 0.0052786	total: 23.1s	remaining: 9.71s
704:	learn: 0.0052729	total: 23.1s	remaining: 9.68s
705:	learn: 0.0052662	total: 23.2s	remaining: 9.64s
706:	learn: 0.0052563	total: 23.2s	remaining: 9.61s
707:	learn: 0.0052473	total: 23.2s	remaining: 9.58s
708:	learn: 0.0052373	total: 23.3s	remaining: 9.55s
709:	learn: 0.0052316	total: 23.3s	remaining: 9.51s
710:	learn: 0.0052209	total: 23.3s	remaining: 9.48s
711:	learn: 0.0052107	total: 23.4s	remaining: 9.45s
712:	learn: 0.0052013	total: 23.4s	remaining: 9.41s
713:	learn: 0.0051951	total: 23.4s	remaining: 9.38s
714:	learn: 0.0051878	total: 23.5s	remaining: 9.35s
715:	learn: 0.0051802	total: 23.5s	remaining: 9.32s
716:	learn: 0.0051725	total: 23.5s	remaining: 9.28s
717:	learn: 0.0051654	total: 23.6s	remaining: 9.25s
718:	learn: 0.0051602	total: 23.6s	remaining: 9.22s
719:	learn: 0.00

862:	learn: 0.0041585	total: 28.3s	remaining: 4.49s
863:	learn: 0.0041511	total: 28.3s	remaining: 4.46s
864:	learn: 0.0041468	total: 28.3s	remaining: 4.42s
865:	learn: 0.0041400	total: 28.4s	remaining: 4.39s
866:	learn: 0.0041373	total: 28.4s	remaining: 4.36s
867:	learn: 0.0041320	total: 28.4s	remaining: 4.32s
868:	learn: 0.0041255	total: 28.5s	remaining: 4.29s
869:	learn: 0.0041221	total: 28.5s	remaining: 4.26s
870:	learn: 0.0041169	total: 28.5s	remaining: 4.23s
871:	learn: 0.0041125	total: 28.6s	remaining: 4.19s
872:	learn: 0.0041086	total: 28.6s	remaining: 4.16s
873:	learn: 0.0041025	total: 28.6s	remaining: 4.13s
874:	learn: 0.0040970	total: 28.7s	remaining: 4.09s
875:	learn: 0.0040903	total: 28.7s	remaining: 4.06s
876:	learn: 0.0040858	total: 28.7s	remaining: 4.03s
877:	learn: 0.0040801	total: 28.8s	remaining: 4s
878:	learn: 0.0040753	total: 28.8s	remaining: 3.96s
879:	learn: 0.0040688	total: 28.8s	remaining: 3.93s
880:	learn: 0.0040647	total: 28.9s	remaining: 3.9s
881:	learn: 0.00

0.9969696969696968

In [32]:
cb_model.save_model('cb_model.cbm')

In [33]:
from lightgbm import Booster
lgbm_model = Booster(model_file='lgbm_model.txt')
cb_model = CatBoostClassifier()
cb_model.load_model('cb_model.cbm')

In [34]:
# 필요한 Mediapipe 솔루션을 초기화합니다.
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# 비디오 캡처 시작
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, img = cap.read()
    if not ret:
        print("Ignoring empty camera frame.")
        continue

    # 이미지 처리
    img = cv2.flip(img, 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result = hands.process(img)
    pose_result = pose.process(img)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    
    hands_data = np.zeros((2, 30))
    
    if result.multi_hand_landmarks:
        for i, hand_landmarks in enumerate(result.multi_hand_landmarks):
            if i>= 2:
                break
            
            angles, joint_coords = calculate_angles(hand_landmarks, img.shape)  # 좌표 데이터도 반환받음
            hands_data[i, :len(angles)] = angles
            data = np.concatenate((data, joint_coords))  # 좌표 데이터를 예측 데이터에 추가
            
    # 데이터 차원 맞추기
    data = np.concatenate((hands_data[0], hands_data[1]))    
    
    if pose_result.pose_landmarks:
        arm_angles = calculate_pose_angles(pose_result.pose_landmarks, img.shape)
        data = np.concatenate((data, arm_angles)) if len(arm_angles) > 0 else data
        
        landmark_data = []
        for idx in [11, 13, 15, 12, 14, 16]:
            landmark = pose_result.pose_landmarks.landmark[idx]
            landmark_data.extend([landmark.x, landmark.y, landmark.z])
            
        data = np.concatenate((data, landmark_data))
        
    data = data.reshape(1,-1)
    predicted_label = cb_model.predict(data)
    print(predicted_label)
    
    # 예측된 라벨을 화면에 표시
    cv2.putText(img, text=str(predicted_label[0]), org=(50, 50),
                fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)
    
    # 손 랜드마크 그리기
    if result.multi_hand_landmarks:
        for res in result.multi_hand_landmarks:
            mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS)
    
    # 화면에 이미지 표시
    cv2.imshow('img', img)
    if cv2.waitKey(30) == 49:  # 1번 키를 누르면 종료
        break
        
cap.release()
cv2.destroyAllWindows()

[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['b

[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['sick']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['belly']]
[['be

[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['down']]
[['sick']]
[['sick']]
[['belly']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['belly']]
[['belly']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick